In [ ]:
import asyncio
import nest_asyncio
import websockets
import json

ip = "192.168.7.2"
port=5555
data_add = "gui_data"
control_add = "gui_control"

ws_control_add = f"ws://{ip}:{port}/{control_add}"
ws_data_add = f"ws://{ip}:{port}/{data_add}"

In [ ]:
nest_asyncio.apply() # event loop needs to be nested - otherwise it conflicts with jupyter's event loop

async def send_msg_callback(ws_address,msg):
    async with websockets.connect(ws_address) as ws:
        await ws.send(json.dumps(msg))
        print(f"Sent: {msg}")

async def rec_msg_callback(ws_address):
    async with websockets.connect(ws_address) as ws:
        while True:
            msg = await ws.recv()
            print(f"Received:{msg}") # TODO filter according to msg type

async def start_listener_callback(ws_address):
    await rec_msg_callback(ws_address)

def send_msg(ws_address,msg):
    loop = asyncio.get_event_loop()
    loop.run_until_complete(send_msg_callback(ws_address,msg))

def start_listener(ws_address):
    loop = asyncio.get_event_loop()
    loop.create_task(start_listener_callback(ws_address)) # create_task() is needed so that the listener runs in the background and prints messages as received without blocking the cell

### connect to control ws

In [ ]:
start_listener(ws_control_add) # this gets a "connection" json response each time, so run only once or TODO filter it out


In [ ]:
send_msg(ws_control_add,  {"watcher":[{"cmd":"hi"}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "unwatch", "watchers":['myvar2']}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd":"list"}]})

### connect to data websockets  

In [ ]:
start_listener(ws_data_add)

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "watch", "watchers":['myvar2']}]})

In [ ]:
send_msg(ws_control_add,{"watcher":[{"cmd": "unwatch", "watchers":['myvar2']}]})

### watcher class

In [ ]:
import asyncio
import nest_asyncio
import websockets
import json
import array
import queue

nest_asyncio.apply() # needed for running event loops inside of jupyter event loop

In [ ]:
class Watcher:

    def __init__(self, ip="192.168.7.2", port=5555, data_add="gui_data", control_add="gui_control"):
        """ Watcher class __summary__.

        Args:
            ip (str, optional): Remote address IP. Defaults to "192.168.7.2".
            port (int, optional): Remote address port. Defaults to 5555.
            data_add (str, optional): Data endpoint. Defaults to "gui_data".
            control_add (str, optional): Control endpoint. Defaults to "gui_control".
        """
        self.ip = ip
        self.port = port
        self.data_add = data_add
        self.control_add = control_add
        self.ws_control_add = f"ws://{self.ip}:{self.port}/{self.control_add}"
        self.ws_data_add = f"ws://{self.ip}:{self.port}/{self.data_add}"
        
        self.ws_control = None
        self.ws_data = None
        
        self.__ctrl_listener = None
        self.__data_listener = None
        
        self.__watcher_vars = [] # only updates when list is called 
        self.__list_response_available = asyncio.Event()
        self.__list_response = None
        
                
        nest_asyncio.apply() # event loop needs to be nested - otherwise it conflicts with jupyter's event loop


    # public methods
    
    def start(self): # TODO check Bela project_name 
        
        
        if self.__ctrl_listener is None: # avoid duplicate listeners
            self.start_ctrl_listener()
        if self.__data_listener is None:
            self.start_data_listener()
        
        # populate watcher vars so that variables received in data stream can be identified
        # TODO test this
        self.__watcher_vars = [var["name"] for var in self.list()["watcher"]["watchers"]]
    
    
    def stop(self):  
        if self.__ctrl_listener is not None:
            self.__ctrl_listener.cancel()
            self.__ctrl_listener = None # empty the listener 
        if self.__data_listener is not None:
            self.__data_listener.cancel()
            self.__data_listener = None
            
    def list(self):
        async def list_coroutine(): # TODO use this template for requesting 
            
            if self.__ctrl_listener is None:
                self.start_ctrl_listener()
            
            self.send_ctrl_msg({"watcher": [{"cmd": "list"}]})
            
            # Wait for the list response to be available
            await self.__list_response_available.wait()
            
            # Now the list response is available
            response = self.__list_response
            self.__list_response_available.clear()  # Reset the event for the next call
            
            return response
                
        return asyncio.run(list_coroutine())

        
    
    def send_ctrl_msg(self, msg):
        self.__send_msg(self.ws_control,self.ws_control_add, msg)
    
    def start_ctrl_listener(self):
        self.__ctrl_listener = self.__start_listener(self.ws_control,self.ws_control_add)
    
    def start_data_listener(self):
        self.__data_listener = self.__start_listener(self.ws_data, self.ws_data_add)


    # __private methods    
    
    ## start listener   

    def __start_listener(self,ws, ws_address):
        loop = asyncio.get_event_loop()
        listener_task = loop.create_task(self.__start_listener_callback(ws,ws_address)) # create_task() is needed so that the listener runs in the background and prints messages as received without blocking the cell
        return listener_task
    
    async def __start_listener_callback(self, ws, ws_address):
        await self.__rec_msg_callback(ws, ws_address)
        
    ## send message
    
    def __send_msg(self, ws, ws_address, msg):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(self.__send_msg_callback(ws,ws_address, msg))   
    
    async def __send_msg_callback(self,ws, ws_address, msg):
        try:
            async with websockets.connect(ws_address) as ws: # here you can use the same websocket for multiple messages -- but avoid using the same for sending and receiving
                await ws.send(json.dumps(msg))
                print(f"Sent: {msg}")
        except Exception as e:
            print(f"Error while sending message: {e}") 
    
    ## receive message     
    
    async def __rec_msg_callback(self, ws, ws_address): # TODO separate control and data messages ??
        try:
            async with websockets.connect(ws_address) as ws:
                
                channel = None 
                
                while True:

                    msg = await ws.recv()
                    
                    if ws_address == self.ws_data_add: # data parsing  ## TODO have these as coroutines 
                        if len(msg) == 3:
                            channel = int(str(msg)[2])
                        elif len(msg)>3: # if array of data, parse to list 
                            _msg = array.array('f', msg).tolist() # TODO name according to channel 
                            
                            if self.__watcher_vars is not None:
                                print(f"{self.__watcher_vars[channel]} :: {_msg}")
                            else:
                                print(f"{msg}")
                    
                    
                    elif ws_address == self.ws_control_add: # control parsing
                        _msg = json.loads(msg)

                        if "watcher" in _msg.keys():
                    
                            # cmd "list" response ## TODO move this as coroutines so that response can be returned 
                            if "watchers" in _msg["watcher"].keys():
                                
                                self.__list_response = _msg["watcher"]["watchers"]
                                self.__list_response_available.set()
                                
                                
                                # # TODO move this to start 
                                # if len(self.__watcher_vars) == 0: # if empty -- only runs after first start -- won't update if variables in watcher change 
                                #     self.__watcher_vars = [var["name"] for var in _msg["watcher"]["watchers"]] # store order for later identifying the variables in the buffer 

                                for var in _msg["watcher"]["watchers"]:
    
                                    print(f'list :: {var["name"]} :: watched: {var["watched"]}, controlled: {var["controlled"]},  value: {var["value"]} ')       
                                             
                                
                    
                    else:
                        print(msg)
                                    

                            
                    # TODO filter according to msg type
                    # TODO identify variable
                    # TODO parsing
        except Exception as e:
            print(f"Error while receiving message: {e}")

    def __del__(self):
        self.stop() # stop websockets

In [ ]:
watcher = Watcher()


In [ ]:
watcher.start()

In [ ]:
watcher.list()

In [ ]:
watcher.send_ctrl_msg({"watcher":[{"cmd": "unwatch", "watchers":['myvar','myvar2']}]})

In [ ]:
watcher.send_ctrl_msg({"watcher":[{"cmd": "list"}]})

In [ ]:
watcher.start()  

In [ ]:

watcher.stop()

In [ ]:
watcher.send_ctrl_msg({"watcher":[{"cmd": "unwatch", "watchers":['myvar','myvar2']}]})
watcher.start()
watcher.send_ctrl_msg({"watcher":[{"cmd": "list"}]}) # ?? guessing the index corresponds to the channel

In [ ]:
watcher2.start()
watcher.start()# calling watcher again does not duplicate prints 

In [ ]:
watcher.stop() # not sure this is working properly
watcher2.stop()

### streamer class

In [ ]:
class Streamer():
    def __init__(self, ip="192.168.7.2", port=5555, data_add="gui_data", control_add="gui_control"):
        """ Streamer class __summary__.

            Args:
                ip (str, optional): Remote address IP. Defaults to "192.168.7.2".
                port (int, optional): Remote address port. Defaults to 5555.
                data_add (str, optional): Data endpoint. Defaults to "gui_data".
                control_add (str, optional): Control endpoint. Defaults to "gui_control".
        """
        self.watcher = Watcher(ip, port, data_add, control_add) # do we need to inherit all methods? or just the send_msg() and start_listener()?
        
    def stream_forever(self,variables=[]):
        
        variables = [variables] if isinstance(variables, str) else variables # check variables is list
        self.watcher.send_ctrl_msg({"watcher":[{"cmd": "watch", "watchers":variables}]})
        self.watcher.start()
        
        
    def stop_streaming(self,variables):  # FIX for some reason this stops streaming all variables
        variables = [variables] if isinstance(variables, str) else variables    
        self.watcher.send_ctrl_msg({"watcher":[{"cmd": "unwatch", "watchers":variables}]})

            
    def stop_streaming_all(self):
        self.watcher.stop()
                                
        

In [ ]:
streamer = Streamer()

In [ ]:
streamer.stream_forever(["myvar1","myvar2"])

In [ ]:
streamer.stop_streaming("myvar2")

In [ ]:
streamer.stop_streaming_all()

In [ ]:
streamer = Streamer() # this should delete the previous watcher so that ws are not duplicated